<a href="https://colab.research.google.com/github/connor735/344_test/blob/main/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem 1: nearest neighbors for data imputation

Real-world datasets often have missing values: this can be due to hardware faults (e.g., a sensor failing), data corruption, ambiguity, or even by design (think about surveys with optional questions). In Python, the special value `NaN` (not-a-number) can be used to indicate a missing value.

Dealing with missing data is not straightforward. One option is to *remove* any sample with missing features $x_{i}$ (or label $y$). However, there are several datasets for which *almost all* samples have one or more missing feature values, and we would be throwing away all the information contained in the non-missing values.

In this problem, we explore the use of the $k$-nearest neighbors algorithm for *data imputation*.

## Question 1

Suppose that you are solving a regression problem where **the only potential missing values are response labels $y^{(i)}$**. Describe a method, based on the $k$-nearest neighbors algorithm, to fill-in missing values based on the available response labels. When do you expect your method to work well, and when do you expect it to produce poor results? Justify your answers and design choices.

## Question 2
Implement your method from Question 1 by completing the provided code snippet in `p1.py` (in the `KnnLabelImputer` class) and **submit it to Gradescope**. You can use the `regression_dataset_with_missing_responses` function, provided in the next code cell, to generate example inputs to help with development.

## Question 3
Now, suppose that we are solving a *classification* problem, and we know that *no labels $y^{(i)}$ are missing*. However, there are now missing *feature* values. Like in **Question 1**, describe a method based on the $k$-nearest neighbors algorithm for performing data imputation on the features of the dataset. You can assume that there are enough samples for each distinct class label. When do you expect your method to work well, and when not to? Justify your answers and design choices.

**Tip**: The nearest neighbors implementation in `scikit-learn` allows you to define your own distance function via the `metric` parameter. You can use this to define a distance function that takes into account missing values and class membership. Think about how you can modify the distance function to favor samples with the same class label.

## Question 4
Implement your method from Question 3 by completing the provided code template in `p1.py` (in the `KnnFeatureImputer` class) and **submit it to Gradescope**. You can use the `classification_dataset_with_missing_features` function from the following cell to generate example inputs to help with development.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.datasets import make_classification


def regression_dataset_with_missing_responses(
    n_samples: int, n_features: int, p_missing: float = 0.1
):
    """
    Generate a regression dataset with missing responses.

    Args:
        n_samples: Number of samples.
        n_features: Number of features.
        p_missing: Probability of missing a response.

    Returns:
        A Pandas DataFrame with features and a response column.
    """
    X, y = make_regression(
        n_samples=n_samples,
        n_features=n_features,
    )
    # Replace response values by NaN with probability p_missing.
    y = np.where(np.random.rand(*y.shape) < p_missing, np.nan, y)
    X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(n_features)])
    return pd.concat([X, pd.Series(y, name="response")], axis=1)


def classification_dataset_with_missing_features(
    n_samples: int, n_features: int, p_missing: float = 0.1, n_classes: int = 3
):
    """
    Generate a classification dataset with missing features.

    Args:
        n_samples: Number of samples.
        n_features: Number of features.
        p_missing: Probability of missing a feature.
        n_classes: Number of classes.

    Returns:
        A Pandas DataFrame with features and a label column.
    """
    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_classes=n_classes,
        n_clusters_per_class=1,
    )
    # Replace feature values in X by NaN with probability p_missing.
    X = np.where(np.random.rand(*X.shape) < p_missing, np.nan, X)
    X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(n_features)])
    return pd.concat([X, pd.Series(y, name="label")], axis=1)

In [ ]:
# Example: regression dataset with 500 samples 20 features, and approximately 10% of responses missing
dataset = regression_dataset_with_missing_responses(500, 10, p_missing=0.1)
dataset.head()

In [ ]:
# Example: classification dataset with 500 samples 10 features, and 10% of responses missing
dataset = classification_dataset_with_missing_features(500, 10, p_missing=0.1)
dataset.head()

# Problem 2: cross-validation for imbalanced datasets

In class, we discussed the use of cross-validation to estimate model performance on unseen data. However, extra care must be taken in cases where our data is *unbalanced*. This is best explained via a cautionary (fictional) tale.


> An aerospace engineer is tasked with creating and fine-tuning a classifier that processes post-flight images of aircraft wings and screens them for the presence of fractures. To train the classifier, they are given access to a dataset of labeled images ($0$ indicates no fracture, $1$ indicates fracture). Following the advice of an LLM, the engineer writes a `scikit-learn` pipeline that:
>
> - pre-processes the data, splits them into $K = 20$ folds using the built-in `KFold` class
> - trains several logistic regression models, one for each fold, and
> - selects the best model based on the estimated (via cross-validation) F1 score.
>
> However, 5 years after they deploy their model, maintenance engineers suggest that it failed to detect several fractures, leading to higher degradation of the aircraft and many more dollars lost. **What went wrong?**


A likely answer is that the dataset is very imbalanced. Specifically, wings are usually structurally intact after flight; unless special care was taken while curating the dataset, most labels will be negative and many folds will not contain any samples from the positive class. Therefore, it is possible that a classifier that always outputs "negative" leads to good F1 score as estimated via cross-validation.

To fix this, we augment K-fold cross validation with the idea of **stratified** sampling. Stratified sampling splits the data in a way that approximately preserves the fraction of class labels within each fold. In `scikit-learn`, this is easy to implement via the `StratifiedKFold` class.

## Question 1

Use the provided function `make_imbalanced_classification_dataset()` below to generate datasets with 500 samples and 50 features. Train a logistic regression classifier using (a) `KFold` and (b) `StratifiedKFold` cross-validation for different values of $K$, and plot the resulting F1 score as a function of the number of folds $K$. Are the results expected or surprising, and why?

> **Note**: you can implement the cross-validation yourself or use the built-in `cross_val_score` function from `scikit-learn` or use the `LogisticRegressionCV` model with the appropriate cross-validation generator.

## Question 2
Another potential approach is to adjust the classification threshold (instead of the default $\frac{1}{2}$). For example, we can make the classifier much more conservative by outputting a positive label after a threshold of $\frac{1}{10}$. Do you think this is a good idea? Why or why not?

In [ ]:
from sklearn.datasets import make_classification
import numpy as np
import pandas as pd


def make_imbalanced_classification_dataset(
    n_samples: int, n_features: int, class_imbalance: float
) -> pd.DataFrame:
    """
    Make an imbalanced classification dataset.

    Args:
        n_samples: The number of samples.
        n_features: The number of features.
        class_imbalance: The proportion of samples assigned to each class.
    """
    if class_imbalance <= 0 or class_imbalance >= 1:
        raise ValueError("class_imbalance must be between 0 and 1")
    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_classes=2,
        weights=[class_imbalance, 1 - class_imbalance],
    )
    X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(n_features)])
    return pd.concat([X, pd.Series(y, name="label")], axis=1)

In [ ]:
dataset = make_imbalanced_classification_dataset(100, 10, 0.1)
dataset.head(10)

# Problem 3: dimension reduction via random projections

In class, we discussed how PCA discovers a few important degrees of freedom of high-dimensional datasets.

The main idea was the following:

> *If data are mostly contained on a low-dimensional subspace, let's discover that subspace and project the data onto it.*

The number of principal components, $k$ reflects our "guess" for the dimension of that subspace.

Another dimension reduction method featuring prominently in theoretical computer science is that of so-called "random projections". Setting the stage, suppose we have $n$ data points living in $d$-dimensional space ($\mathbb{R}^d$ for simplicity). The random projections method proceeds as follows:

- Generate $k$ vectors, $g^{(1)}, \dots, g^{(k)}$, randomly (often according to a Gaussian distribution).
- Arrange these vectors as the columns of a matrix:

$$
\Pi := \begin{bmatrix}
	g^{(1)} & \cdots & g^{(k)}
\end{bmatrix} \in \mathbb{R}^{d \times k}.
$$

- For each of the $n$ points in your dataset, say $x^{(1)}, \dots, x^{(n)}$:
	- Compute the product $\widetilde{x}^{(i)} = \Pi^{\top} x^{(i)}$.
- This results in a matrix $\widetilde{X} \in \mathbb{R}^{n \times k}$ containing the "reduced" dataset.
- Downstream tasks (e.g., regression or classification) can be performed on the reduced dataset instead.
    - Any new data point $x$ can be projected to $\widetilde{x} := \Pi^{\top} x$.


The random projections method works because the matrix $\Pi$ produces "low distortion": for all pairs $i \neq j$,

$$
(1 - \varepsilon) \|x^{(i)} - x^{(j)}\| \leq \| \widetilde{x}^{(i)} - \widetilde{x}^{(j)}\| \leq (1 + \varepsilon) \|x^{(i)} - x^{(j)}\|,
$$

whenever the number of the random vectors $k$ satisfies the following inequality:

$$
k \geq \frac{10 \log(n)}{\varepsilon^2}.
$$

> Note that the ambient dimension, $d$, **does not appear** in the bound at all! This result is known in the literature as the *Johnson-Lindenstrauss lemma*.

In this exercise, you will explore the use of random projections with 2 different variants: one where the random vectors are dense, and one where the random vectors are *sparse* (i.e., mostly zeros). Then, you will compare the quality of the resulting embeddings with that provided by principal component analysis.

## Question 1

Download the code template `p3.py`. You will find two functions with the following code signatures:

```python
def apply_sparse_random_projections(dataset: ArrayLike, num_projections: int) -> ArrayLike:
	...

def apply_dense_random_projections(dataset: ArrayLike, num_projections: int) -> ArrayLike:
	...
```

These functions accept a `dataset` (e.g., a Numpy table) where each row is a sample and the number of projections $k$, `num_projections`. They should return the reduced dataset after applying the random projections method. Implement both methods and submit your code to Gradescope.

> **Tip**: Use the `SparseRandomProjection` and `GaussianRandomProjection` classes from `scikit-learn`. You will have to instantiate them with the appropriate arguments.

## Question 2

Use the provided snippet below to create a high-dimensional dataset. Using your implementation from Question 1, compute the following quantities for $k = 2^{j}$ ($j = 2, \dots, 10$):

- **Maximum and minimum distortion**: the min and max ratio of distances between projected and original data points.

  $$
  \gamma_{\mathsf{upper}} := \max_{i \neq j} d_{i, j} \;\; \text{and} \;\; \gamma_{\mathsf{lower}} := \min_{i \neq j} d_{i, j},
  $$
  
  where we define the ratios $d_{i,j}$ as follows:
  
  $$
  \frac{\|\widetilde{x}^{(i)} - \widetilde{x}^{(j)}\|}{\|x^{(i)} - x^{(j)}\|}.
  $$

- The ratio $\sqrt{\frac{\lceil \log n \rceil}{k}}$.

Make a plot of these quantities as a function of $k$ (once for sparse and once for dense random projections). In your narrative writeup, discuss the following questions:

- Is the behavior of these quantities surprising as you increase $k$? Why or why not?
- Is the behavior of these quantities surprising as you move from sparse to dense random projections? Why or why not?
- Would you recommend this method for a huge dataset (very large $n$) of relatively small dimension ($d$)? Why or why not?

## Question 3

For the same dataset, compare the distortion factors from Question 1 with those attained by PCA (using $k$ as the number of  principal components); do this **for all** $k$ you tried in Question 1 and make a shared plot depicting the distortion factors for both methods as a function of $k$.

- Are the distortion factors from PCA higher or lower?
- Which distortion factor did you expect to be higher before you completed the question, and why?

In [ ]:
# Code to create a high-dimensional dataset
import pandas as pd
import numpy as np
from sklearn.datasets import make_friedman1


def create_high_dim_dataset(n_samples: int, n_features: int) -> pd.DataFrame:
    X, y = make_friedman1(
        n_samples=n_samples,
        n_features=n_features - 1,
        noise=0.1,
        random_state=42,
    )
    return pd.DataFrame(np.hstack([X, y.reshape(-1, 1)]))


# Example: dataset with 128 samples, 2048 features, and some noise.
X = create_high_dim_dataset(128, 2048)

# Problem 4: Kaggle competition

In this problem, you will work on the [Diabetes Health Indicators dataset](https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset) from Kaggle. The dataset comprises responses to a CDC survey. The code in the cell below splits the dataset into features and labels, resulting in a binary classification task. For this problem, your only deliverable should be a Jupyter notebook that implements all the necessary code and addresses the following questions:

## Question 1

The dataset contains numeric, nominal and ordinal features. Which features fall into which category, and how should we preprocess each category?

## Question 2

Train a classifier for the binary classification task, and report the average accuracy and Area-Under-Curve (AUC) using 5-fold cross-validation. Does this problem warrant using additional metrics, such as the F1 score, in addition to accuracy? Why or why not?

**Note**: You are allowed to choose any model suitable for binary classification you want, in addition to those you have already seen in class. You are welcome to peruse the scikit-learn documentation for additional models, if you want to try them out.

## Question 3

If you wanted to further improve the performance of your classifier for this problem, what additional methods / model improvements would you consider and why?

In [ ]:
import kagglehub
import os
import pandas as pd

# Download latest version
path = os.path.join(
    kagglehub.dataset_download("mohankrishnathalla/diabetes-health-indicators-dataset"),
    "diabetes_dataset.csv",
)
df = pd.read_csv(path)

# Define the feature and labels
# We want to solve the binary classification task, drop the other two labels
features = df.drop(columns=["diabetes_risk_score", "diabetes_stage", "diagnosed_diabetes"])
labels = df["diagnosed_diabetes"]

In [ ]:
features.head()

In [ ]:
labels.head()